In [ ]:
%pip install openai langchain tiktoken wikipedia langchain-experimental langchainhub docarray -q 

# Application Development

## Quick Overview

### Prompts

Prompts are short instructions or phrases used to guide a language model like a Machine Learning Language Model (LLM) on what type of text to generate. These instructions can be as simple as a word or phrase, or they can be complete paragraphs, depending on the desired response type and the specific model being used.

### Langchain

`LangChain` is a framework for developing applications powered by language models. It enables applications that:

- Are context-aware: Connect a language model to sources of context (prompt instructions, few-shot examples, content to ground its response in, etc.).
- Reason: Rely on a language model to reason (about how to answer based on provided context, what actions to take, etc.).



<!-- 
### Langchain: Question and Answer with DocArrayInMemorySearch

Anteriormente se uso Chroma para almacenar la base de datos, ahora se usara `DocArrayInMemorySearch` para hacer las consultas, el procedimiento es similar -->


<!-- #### 

- Question and Answer
  - leer un documento
  - guardar el documento en una base de datos pasando previamente por un modelo de embeddings
  - hacer las consultas
  - la base de datos retorna una lista de elementos que segun la coincidencia de vectoores, son importantes para responder la consulta
  - los documentos se pasan por un motor de llm para sintetizar y generar una respuesta adecuada  -->



## Model Parsel

### Chat API: Open AI

- [Open ai version 1.0.0](https://github.com/openai/openai-python/discussions/742)

On the internet, several tutorials can be found on how to use the OpenAI API, but if these tutorials are dated before November, they might be outdated. This is because installing `pip install openai` will provide a version of 1.x.x, whereas tutorials before that date were working with the beta version.

For example, to access the OpenAI chat, note in the following example that the new model of creation is based on creating an instance of the `OpenAI()` module instead of having it globally, and to access the methods we switch from `openai.ChatCompletion` to `OpenAI().chat.completions`:


```python
# old
import openai

completion = openai.ChatCompletion.acreate(model="gpt-3.5-turbo", messages=[{"role": "user", "content": "Hello world"}])

# new
from openai import OpenAI

client = OpenAI()
completion = client.chat.completions.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": "Hello world"}])
```

Regarding responses, it relies on [`pydantic models`](https://docs.pydantic.dev/latest/concepts/models/), meaning that dictionaries are no longer used, but rather attributes. However, `pydantic` allows conversion to a dictionary with `model.dict()`:

```python

# old 
import json
import openai

completion = openai.Completion.create(model='gpt-3.5-turbo', messages=[{}])
print(completion['choices'][0]['text']) # /// old
print(completion.get('usage')) #  /// old
print(json.dumps(completion, indent=2))

# new
from openai import OpenAI

client = OpenAI()

completion = client.completions.create(model='gpt-3.5-turbo', messages=[{}])
print(completion.choices[0].message.content)  #/// new
```

First, we need to call the class, create the OpenAI client, and pass our api_key as a parameter. However, if we are using environment files `.env`, we only need to have the variable name correctly set, in this case `OPENAI_API_KEY`. In which we can define our api_key.

![](https://imgs.search.brave.com/j6GBXNcpKlQxBOvfNFyojLGZjzHPUAYi71MoT5kmFes/rs:fit:860:0:0/g:ce/aHR0cHM6Ly9sYWJz/LnRoaW5rdGVjdHVy/ZS5jb20vc3RvcmFn/ZS9pbWFnZS01LnBu/Zw)

To access the variable, we must run the following code and call our environment variable.

In [ ]:
from google.colab import userdata
import os
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

To directly access our response, a function is created which takes the question (or prompt) and the model we want to use as parameters, emphasizing how responses from pydantic are now used. Therefore, we must access the response with the attributes, in this case `completion.choices[0].message.content`. Let's try with a general question and a prompt.

In [ ]:
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": "How do I output all files in a directory using Python?",
        },
    ],
)
print(completion.choices[0].message.content)  

### Open AI and LangChain

Fortunately, LangChain updates its code as its dependencies do, so in the latest versions of LangChain, this migration is already implemented.

To use this way of calling OpenAI, we can replicate the previous example with prompts. First, we call the `ChatOpenAI` module and define the prompt with the variables we want, and save it in the `message` variable.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.0, model=llm_model)
# chat # callable([list])

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""
prompt_template = ChatPromptTemplate.from_template(template_string)
variable1 = """American English \
in a calm and respectful tone
"""
variable2 = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

message = prompt_template.format_messages(style=variable1, text=variable2)

Finally, we get the response.

In [ ]:
response = chat(message)
print(response.content)

## Output Parser

In some cases, you may want the response type to have a certain structure, such as JSON. However, when we obtain the response from LangChain, we notice that the result is a string and we cannot access its elements.

Fortunately, we can convert the output string to a Python dictionary. First, we import `ResponseSchema` and `StructuredOutputParser`. Then, we create the elements that we want to extract with a description so that the model knows what to parse and format as required. Finally, we pass this through the format instructions within the initial prompt.

In [ ]:
# Base, repeat steps
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)

messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0, model=llm_model)
response = chat(messages)

print('Content', response.content)
print("type", type(response.content))
# It's just a String

Parse Outputs

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

# Define the Schemas, with descriptions
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

# Define the method into a `output` parser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

In [ ]:
# Run LLm
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)
response = chat(messages)
print("content", response.content)
print("type", type(response.content))

When the model is run, the response obtained is as per the format instructions provided. We then pass it through the `parse` method of the content, which returns the Python dictionary.

In [ ]:
# Get dictionary

result_dict = output_parser.parse(response.content)
result_dict

## LangChain Memory

When interacting with `ChatGPT`, the responses from one interaction to the next are often related because `ChatGPT` contextualizes its responses, referencing previous inputs and outputs to formulate the latest response.

![](https://python.langchain.com/assets/images/memory_diagram-0627c68230aa438f9b5419064d63cbbc.png)

To simulate this scenario with LangChain, we utilize four types of memory:

- ConversationBufferMemory
- ConversationBufferWindowMemory
- ConversationTokenBufferMemory
- ConversationSummaryMemory
<!-- - **ConversationBufferMemory**: Records and manages previous parts of a conversation, allowing language models to maintain conversational context. It's crucial for applications like chatbots to ensure a seamless conversational flow.

- **ConversationBufferWindowMemory**: Retains a window of recent conversation exchanges, limiting memory growth. Useful for keeping track of recent terms without accumulating excessive memory.

- **ConversationTokenBufferMemory**: Manages conversation tokens, controlling memory based on token limits. Helps manage costs associated with token-based pricing for language model calls.

- **ConversationSummaryMemory**: Summarizes conversation history using an LLM, providing a concise overview of past exchanges. Useful for condensing lengthy conversations into manageable summaries, aiding memory storage efficiency. -->

To use these methods, we can refer to the following example:

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChains

llm_model = 'gpt-3.5-turbo'
llm = ChatOpenAI(temperature=0.0, model=llm_model)

```python
# Reference only
from langchain.memory import Conversation{Method}Memory as Memory

llm = ChatOpenAI(temperature=0.0, model='gpt-3.5-turbo')
memory = Memory()
conversation = ConversationChains(
    llm=llm,
    memory=memory
)

conversation.predict(input='Question')  # Add memory (input, output)
memory.load_memory_variables({})  # Returns memory (history)
memory.save_context({"input": "user_content", "output": "AI response"})  # Add manual memory
```


### ConversationBufferMemory

Using this in a chain (setting verbose=True so we can see the prompt).

In [ ]:
from langchain.memory import ConversationBufferMemory as Memory 

memory = Memory()
conversation = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True
)

conversation.predict(input="Hi there!")

In [ ]:
conversation.predict(input="Tell me about yourself.")

In [ ]:
conversation.load_memory_variables({})

### ConversationBufferWindowMemory

`ConversationBufferWindowMemory` keeps a list of the interactions of the conversation over time. It only uses the last K interactions. This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory as Memory 

memory = Memory()
conversation = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True
)

memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

memory.load_memory_variables({})

### ConversationTokenBufferMemory

`ConversationTokenBufferMemory` keeps a buffer of recent interactions in memory, and uses token length rather than number of interactions to determine when to flush interactions.

In [ ]:
from langchain.memory import ConversationTokenBufferMemory as Memory 

memory = Memory(max_token_limit=60)

conversation = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=False
)

conversation_with_summary.predict(input="Hi, what's up?")
conversation_with_summary.predict(input="Just working on writing some code!")

### ConversationSummaryMemory

`ConversationSummaryBufferMemory` combines the two ideas. It keeps a buffer of recent interactions in memory, but rather than just completely flushing old interactions it compiles them into a summary and uses both. It uses token length rather than number of interactions to determine when to flush interactions.

In [ ]:
from langchain.memory import ConversationSummaryMemory as Memory

schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = Memory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
memory.save_context(
    {"input": "What is on the schedule today?"}, {"output": f"{schedule}"}
)

conversation = ConversationChain(llm=llm, memory=memory, verbose=False)

conversation.predict(input="What would be a good demo to show?")
memory.load_memory_variables({})

## LangChain: Chains

Previously, we used chains, but in a quick manner to obtain a response from the LLM model.

Chains in LangChain encompass sequences of function calls, whether to an LLM, a tool, or a data preprocessing step. LCEL (LangChain Execution Language) is the primary method for constructing these chains, offering both custom and off-the-shelf options.

Combine multiple chains where the output of one chain is the input of the next chain.

- **LLMchain**: It's just the combination of the LLM and the prompt (simple chain).


- **Simple Sequential Chain**: A chain where one chain's output is another chain's input, resulting in a final output.

<img src="https://miro.medium.com/v2/resize:fit:1400/1*sYdb7ca9vcmDV0gOiNaruQ.png" width="600px" height="300px">

- **Sequential Chain**: Similar to the simple sequential chain but allows multiple input variables for each step, useful for more complex downstream chains.

![](https://av-eks-lekhak.s3.amazonaws.com/media/__sized__/article_images/7_Bpn6EWD-thumbnail_webp-600x300.webp)

- **Router Chain**: Utilizes an LLM to route between potential options, deciding which sub-chain to pass input to based on input characteristics.

<img src="https://miro.medium.com/v2/resize:fit:700/1*0TDSAfaL2Q46TnFWkQTagg.jpeg" width="600px" height="300px">

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate as CPrompt
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.chains.router import MultiPromptChain, LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

llm_model = 'gpt-3.5-turbo'
llm = ChatOpenAI(temperature=0.9, model=llm_model)

### LLM chain:

In [ ]:
prompt = CPrompt.from_template("What is the best name to describe \
    a company that makes {product}?")
chain = LLMChain(llm=llm, prompt=prompt)
input_ = "Magic wands in the Harry Potter universe"
chain.run(input_)

### SimpleSequentialChain

In [ ]:
first_prompt = CPrompt.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)
first_chain = LLMChain(llm=llm, prompt=first_prompt)
second_prompt = CPrompt.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
second_chain = LLMChain(llm=llm, prompt=second_prompt)

simple_chain = SimpleSequentialChain(
    chains=[first_chain, second_chain],
    verbose=True
)

input_1 = "Magic wands in the Harry Potter universe"

simple_chain.run(input_1)

### SequentialChain

In [ ]:
from langchain.chains import SequentialChain
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(
    llm=llm, prompt=first_prompt, 
    output_key="English_Review"
    )

second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [ ]:
review = "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"

# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True,
)

overall_chain(review)

### Router Chain: Multi-Prompt Router

#### Contextual Inputs and Prompts

In [ ]:
# Templates for different domains
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""

math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""

computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

# Information about each prompt
prompt_infos = [
    {"name": "physics", "description": "Good for answering questions about physics", "prompt_template": physics_template},
    {"name": "math", "description": "Good for answering math questions", "prompt_template": math_template},
    {"name": "History", "description": "Good for answering history questions", "prompt_template": history_template},
    {"name": "computer science", "description": "Good for answering computer science questions", "prompt_template": computerscience_template}
]

#### Making the Prompt Destination Templates

In [ ]:
# Create chains for each prompt template
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
# Create descriptions for prompt destinations
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

# Default prompt and chain
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

#### Multi-Prompt Router Template

In [ ]:
# Template for multi-prompt router
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
    ```json
    {{
        "destination": string \ name of the prompt to use or "DEFAULT",
        "next_input": string \ a potentially modified version of the original input
    }}
    \```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_input" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

#### Creating the Final Router Prompt and Chain

In [ ]:
# Create the router prompt template
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

# Create the router chain
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

#### Create the Multi-Prompt Chain

In [ ]:
# Create the multi-prompt chain
chain = MultiPromptChain(
    router_chain=router_chain, 
    destination_chains=destination_chains, 
    default_chain=default_chain, 
    verbose=True
)

#### Test the Interpreter

In [ ]:
# Test for Physics Interpreter
chain.run("What is black body radiation?")

In [ ]:
# Test for Math Interpreter
chain.run("what is square of (16)")

In [ ]:
# Not related / Biology
chain.run("Why does every cell in ourt body contain DNA?")

For more details consult [langchain Chains API](https://python.langchain.com/docs/modules/chains)


## Agents: Toolkits and Wikipedia

### Introduction to Agents

Natural Language Models (NLMs) like GPT-3.5 often rely on preprocessed data up to a certain time, typically until 2022 for GPT-3.5. However, continuous knowledge creation or prompt-specific information requires specific tools to utilize this information and synthesize it. Essentially, agents gather various pieces of information and synthesize relevant content, potentially consulting web sources and other tools to respond to queries effectively.

### Wikipedia Tool

When a query is made to the language model endpoint and it doesn't find relevant information, the "tools" will search for information in other specified sources to generate context and responses to the question. To simulate this hypothetical case using information found on Wikipedia, we can utilize the `tools` components of LangChain. Specifically, to integrate Wikipedia functionality, we can consult more tools in [`Components.tools`](https://python.langchain.com/docs/integrations/tools).

In [ ]:
# Import necessary methods and functions
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

# Define the language model
llm_model = 'gpt-3.5-turbo'
llm = ChatOpenAI(temperature=0, model=llm_model)

# Load Wikipedia tool into a list of tools
tools = load_tools(["wikipedia"], llm=llm)

# Initialize the Wikipedia agent
wikipedia = initialize_agent(
    tools,
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True  # Show verbose output
)

*Examples*

In [ ]:
# Query about the current president of Peru
wikipedia("who is the current president of Peru?")

In [ ]:
# Query about the death of Sebastian Piñera, former President of Chile
wikipedia("Sebastian Piñera die? cause of death")

This code sets up an agent to interact with Wikipedia for retrieving information. It initializes the agent with Wikipedia as one of the tools to consult when the language model endpoint doesn't have relevant information. Then, it demonstrates how to use the agent to query information, such as the current president of Peru and details about the death of Sebastian Piñera, the former President of Chile.

## Python REPL Tool

The Python REPL (Read-Eval-Print Loop) tool allows for the execution of Python code based on the questions asked, providing solutions accordingly.

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor
from langchain_experimental.tools import PythonREPLTool
from langchain.agents import create_openai_functions_agent

# Define instructions for the agent
instructions = """You are an agent designed to write and execute Python code to answer questions.
You have access to a Python REPL, which you can use to execute Python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
"""
# Pulling a base prompt from Hub
base_prompt = hub.pull("langchain-ai/openai-functions-template")
# Partially filling the base prompt with instructions
prompt = base_prompt.partial(instructions=instructions)

# Initialize the Python REPL tool
tools = [PythonREPLTool()]

# Create an agent using OpenAI functions
agent = create_openai_functions_agent(ChatOpenAI(temperature=0), tools, prompt)
# Initialize an AgentExecutor for invoking the agent
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

*Example*

In [ ]:
# Querying the agent for the 10th Fibonacci number
agent_executor.invoke({"input": "What is the 10th Fibonacci number?"})

In [ ]:
# Sorting a list of customers by last name and then first name
customer_list = [
    ["Harrison", "Chase"],
    ["Lang", "Chain"],
    ["Dolly", "Too"],
    ["Elle", "Elem"],
    ["Geoff", "Fusion"],
    ["Trance", "Former"],
    ["Jen", "Ayai"],
]
agent_executor.invoke(
    {
        "input": f"""
        Sort these customers by last name and then first name and print the output: {customer_list}
        """
    }
)